This is a copy of Benjamin Sorensen's blog post about using redist, which you can find here: https://www.benjaminsorensen.me/post/mt-redistricting/

In [4]:
library(sf)
library(ggplot2)
library(tidyverse)
library(tibble)
library(magrittr)
library(redist)
library(sp)
library(spData)
library(spdep)
library(igraph)


Attaching package: 'magrittr'


The following object is masked from 'package:purrr':

    set_names


The following object is masked from 'package:tidyr':

    extract


To access larger datasets in this package, install the spDataLarge
package with: `install.packages('spDataLarge',
repos='https://nowosad.github.io/drat/', type='source')`


Attaching package: 'igraph'


The following objects are masked from 'package:dplyr':

    as_data_frame, groups, union


The following objects are masked from 'package:purrr':

    compose, simplify


The following object is masked from 'package:tidyr':

    crossing


The following object is masked from 'package:tibble':

    as_data_frame


The following objects are masked from 'package:stats':

    decompose, spectrum


The following object is masked from 'package:base':

    union




In [5]:
va_2015_file <- "C:/Users/madie/OneDrive/data/pre-redist/VA_precinct_2015/VA_precinct_2015.shp"
df <- st_read(va_2015_file)

Reading layer `VA_precinct_2015' from data source `C:\Users\madie\OneDrive\data\pre-redist\VA_precinct_2015\VA_precinct_2015.shp' using driver `ESRI Shapefile'
Simple feature collection with 2577 features and 14 fields
geometry type:  MULTIPOLYGON
dimension:      XYZ
bbox:           xmin: 9963623 ymin: 6151696 xmax: 12442960 ymax: 7217011
z_range:        zmin: 0 zmax: 0
projected CRS:  NAD83 / Virginia North (ftUS)


In [6]:
adjlist <- spdep::poly2nb(df, queen = FALSE)

In [7]:
# much faster for st_distance to use centroids rather than polygons
distmat <- 
  df %>% 
  mutate(centroid = st_centroid(geometry)) %>% 
  st_set_geometry(value = "centroid") %>% 
  select(-geometry) %>% 
  st_distance()

In [ ]:
alg_mcmc <- redist.mcmc(
  adjobj = adjlist,
  popvec = df$pop,
#  initcds = initcds$cd_ns, I dont' have initial districts I'm starting with
  maxiterrsg = 1e8,
  ndists = 11,
  nsims = 10000,
  constraint = c("population", "compact"),
  constraintweights = c(1, 1),
  popcons = .01,
  ssdmat = distmat,
  savename = "redist.mcmc"
) %>% 
  invisible()